# Trabalho Prático 1 - Grupo 15

#### João Gonçalves - pg46535
#### Sara Queirós - pg47661
## Exercício 3 

A segurança de kyber é baseada da dificuldade de resolver problemas com LWE. Kyber constrói-se em 2 fases:
IND-CPA para Kem de 32 bytes 

In [6]:
import math, os, numpy as np
from hashlib import shake_128 as XOF, sha3_256 as H, sha3_512 as G, shake_256 as KDF
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing

#Parâmetros definidos para implementação do kyber
n = 256
n_linha = 9
q = 3329
Q = PolynomialRing(GF(q), 'w')
y = Q.gen()
RQ = QuotientRing(Q, y^n+1)

def PRF(s, b):
    kdf = KDF()
    kdf.update(s+b)
    return kdf

def modMM(a,b):
    r = a % b
    #Testar se b é par
    if (mod(b,2)==0):
        inf , sup = -b/2, b/2 #Calculo dos limites sup e inf
    else:
        inf , sup = (-b-1)/2, (b-1)/2
    #Temos de garantir que o módulo se encontra no intervalo estipulado
    while r > sup:
        r -= b
    while r < inf:
        r+=b
    return r 

#Função Parse que recebem uma stream de bytes 
# e devolve a sua representação NTT
def parse(b):
    coefs = [0] * n
    i = 0
    j = 0
    while j < n:
        d1 = b[i]+256*(modMM(b[i+1], 16))
        d2 = b[i+1]/16 +16*b[i+2]
        if d1 < q:
            coefs[j] = d1
            j=j+1
        if d2<q and j < n:
            coefs[j] = d2
            j=j+1
        i += 3
    return RQ(coefs)

#Converte array de bytes em bits
def BytestoBits(byteArr):
    bitArr = []
    for elem in byteArr:
        eachByte = []
        for i in range(0,8):
            eachByte.append((elem // 2**((i%8)) % 2))
        
        for i in range(0,len(eachByte)):
            bitArr.append(eachByte[i])
    return bitArr

#Converte array de bits em bytes
def BitsToBytes(bits):
    byteArr = []
    nr_bytes = int(len(bits)/8)

    for i in range(nr_bytes):
        elem = 0
        for j in range(8):
            elem += (int(bits[i*8+j]) * 2**j)
        byteArr.append(elem)

    return byteArr



b = "Minhastring funhhhhhhhhhcionou".encode('utf-8')
bb = BytestoBits(b)
bbb = BitsToBytes(bb)
print(bytes(bbb).decode('utf-8'))

def cbd(noise, byteArray):
    bits = BytestoBits(byteArray)
    f = []
    for i in range(256):
        a = 0
        b = 0
        for j in range(n):
            a += bits[2*i*noise + j]
            b += bits[2*i*noise + noise + j]
        f.append(a-b)
    return RQ(f)

def decode(l, byteArray):
    f=[]
    bits = BytestoBits(byteArray)
    for i in range(256):
        elem = 0
        for j in range(l):
            elem += int(bits[i*l+j] * 2**j)
        f.append(elem)
    return RQ(f)

def encode(l, polinomiais):
    bits = []
    coefs = polinomiais.list()
    for i in range(256):
        elem = int (coefs[i])
        for j in range(l):
            bits.append(elem%2)
            elem = elem // 2
    return BitsToBytes(bits)
        



Minhastring funhhhhhhhhhcionou


In [7]:
#Testes ao encode/decode
arr = [32,42,34,5,35,3,5,7,54,34,21,43,0,0,0,32,42,34,5,35,3,5,7,54,34,21,43,5,2,5,2,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,46,7,3,3,21,43,53,0,0,0,0,0,0,0,0,0,0,0]
polinomios = decode(2,arr)
print(encode(2,polinomios))
if arr == encode(2,polinomios) :
    print('\n Os arrays correspondem!')
else :
    print('\n Arrays não correspondem')

[32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 0, 0, 0, 32, 42, 34, 5, 35, 3, 5, 7, 54, 34, 21, 43, 5, 2, 5, 2, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 46, 7, 3, 3, 21, 43, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

 Os arrays correspondem!


In [8]:
#classe NTT - Numeric-Theoretic tranform, fornecido pelo professor 
class NTT(object):
#    
    def __init__(self, n=16, q=None, base_inverse=False):
        if not  n in [16,32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        x = g.roots(multiplicities=False)[-1]
        self.x = x
        if  base_inverse:
            rs = [x^(2*i+1)  for i in range(n)] 
            self.base = crt_basis([(w - r) for r in rs]) 
        else:
            self.base = None

    
    
    def ntt(self,f,inv=False):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u))      
            
        def _ntt_(x,N,f,inv=inv):
            if N==1:
                return f
            N_ = N//2 ; z =  x^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(z,N_,f0,inv=inv) ; ff1 = _ntt_(z,N_,f1,inv=inv)  
    
            s  = self.F(1) if inv else x 
            ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * z                    
            return ff 

        
        vec = _expand_(f)
        if  not inv:
            return self.R(_ntt_(self.x,self.n, vec, inv=inv))
        elif self.base != None:
            return sum([vec[i]*self.base[i] for i in range(self.n)])
        else:
            n_ = (self.F(self.n))^-1
            x_ = (self.x)^-1 
            u = _ntt_(x_,self.n,vec, inv=inv)
            
            return self.R([n_ * x_^i * u[i] for i in range(self.n)])
 
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)


In [9]:
# Teste
N=1024

#T = NTT(n=N,q=343576577,base_inverse=False)
T = NTT(n=N,base_inverse=False)

f = T.random_pol(N//2)
print(f)

ff = T.ntt(f)
#print(ff)

fff = T.ntt(ff,inv=True)

#print(fff)
print("Correto ? ",f == fff)

310*w^512 + 7596*w^511 + 9914*w^510 + 3194*w^509 + 5379*w^508 + 7205*w^507 + 10932*w^506 + 2835*w^505 + 6157*w^504 + 2886*w^503 + 9474*w^502 + 683*w^501 + 1048*w^500 + 3921*w^499 + 2225*w^498 + 1319*w^497 + 5828*w^496 + 5272*w^495 + 5881*w^494 + 5579*w^493 + 11440*w^492 + 9588*w^491 + 7865*w^490 + 7296*w^489 + 4139*w^488 + 406*w^487 + 550*w^486 + 5833*w^485 + 1867*w^484 + 1215*w^483 + 3847*w^482 + 4637*w^481 + 3954*w^480 + 525*w^479 + 11171*w^478 + 11589*w^477 + 9717*w^476 + 121*w^475 + 3852*w^474 + 5689*w^473 + 7600*w^472 + 5103*w^471 + 3099*w^470 + 10263*w^469 + 10979*w^468 + 4173*w^467 + 7498*w^466 + 2290*w^465 + 4824*w^464 + 2340*w^463 + 11997*w^462 + 6734*w^461 + 4873*w^460 + 10865*w^459 + 8205*w^458 + 2969*w^457 + 1321*w^456 + 11450*w^455 + 2657*w^454 + 11794*w^453 + 772*w^452 + 6303*w^451 + 11966*w^450 + 8845*w^449 + 2804*w^448 + 2853*w^447 + 1190*w^446 + 11147*w^445 + 61*w^444 + 5427*w^443 + 7965*w^442 + 4328*w^441 + 4076*w^440 + 11709*w^439 + 7578*w^438 + 9803*w^437 + 3007*w^4

In [10]:
class Kyber:
    def __init__(self):
        self.n = 256
        self.n_linha = 9
        self.k = 2
        self.q = 3329
        self.n1 = 3
        self.n2 = 2 
        self.du = 10 
        self.dv = 4
        self.d = 2**-139
        
        
        
    def keygen(self):
        d = os.urandom(32)
        h = G(d)
        digest = h.digest()
        ro,sigma = digest[:32], digest[32:]
        N = 0
        A = []
        #Construção da Matriz A no domínio NTT
        for i in range(self.k):
            A.append([])
            for j in range(self.k):
                xof = XOF()
                xof.update(ro + j.to_bytes(4, 'little') + i.to_bytes(4, 'little'))
                parsed = parse(xof.digest(int(q)))
                A[i].append(parsed)
        #Construção do vetor s
        s=[]
        for i in range(self.k):
            prf = PRF(sigma, int(N).to_bytes(4, 'little'))
            s.append(cbd(self.n1, prf.digest(int(self.q))))
            N += 1
        #Construção do vetor e
        e=[]
        for i in range(self.k):
            prf = PRF(sigma, int(N).to_bytes(4, 'little'))
            e.append(cbd(self.n1, prf.digest(int(self.q))))
            N += 1
            
        #Calculo do NTT de s
        T = NTT()
        i = 0
        print(s[1])
        T.ntt(s[i])
    
K = Kyber()
K.keygen()


wbar^255 + 3327*wbar^254 + 3327*wbar^253 + 3328*wbar^252 + wbar^251 + 3328*wbar^250 + 2*wbar^249 + wbar^248 + wbar^247 + wbar^244 + 3328*wbar^243 + 2*wbar^242 + 3328*wbar^240 + wbar^239 + 3328*wbar^238 + wbar^237 + wbar^236 + wbar^234 + 3328*wbar^231 + 2*wbar^229 + wbar^228 + wbar^227 + 3328*wbar^225 + 3328*wbar^224 + 3328*wbar^223 + 3328*wbar^222 + 3328*wbar^221 + 2*wbar^219 + 2*wbar^218 + wbar^216 + wbar^215 + 3328*wbar^210 + wbar^209 + 3327*wbar^208 + 3327*wbar^206 + wbar^204 + wbar^203 + wbar^202 + 3328*wbar^200 + wbar^198 + wbar^197 + 3328*wbar^195 + 3328*wbar^193 + 3328*wbar^191 + 3327*wbar^190 + wbar^189 + 3327*wbar^188 + 3328*wbar^186 + 3328*wbar^184 + 3*wbar^183 + wbar^182 + 3328*wbar^179 + 3327*wbar^178 + 3326*wbar^176 + 2*wbar^174 + wbar^173 + 2*wbar^172 + wbar^171 + 2*wbar^168 + 3328*wbar^167 + 3328*wbar^166 + wbar^165 + wbar^163 + 3327*wbar^161 + wbar^160 + 3328*wbar^159 + 3328*wbar^156 + 3328*wbar^155 + 3327*wbar^153 + 2*wbar^152 + 3326*wbar^151 + 3328*wbar^150 + 3328*wba

TypeError: unsupported operand parent(s) for *: 'Finite Field of size 97' and 'Finite Field of size 3329'